In [1]:
import os
import math
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Flatten, Lambda, Dropout, Activation, LSTM, GRU, \
        TimeDistributed, Convolution1D, MaxPooling1D, Convolution2D, MaxPooling2D, \
        BatchNormalization, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, \
        ZeroPadding2D, Reshape, merge, GlobalAveragePooling2D, GlobalMaxPooling2D,AveragePooling2D
from keras.layers.local import LocallyConnected1D
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K
from keras.models import Model
from keras.models import load_model  
from sklearn.model_selection import train_test_split
# from skimage.transform import resize
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
%matplotlib inline

Using TensorFlow backend.


In [ ]:
def get_crnn_model_koz4k(input_shape, n_conv_blocks, n_conv_filters, conv_filter_size, n_lstm_blocks, n_lstm_units):
    model_input = Input(input_shape, name='input')
    layer = model_input
    #convolution blocks
    for i in range(n_conv_blocks):
        layer = Convolution1D(
                filters=n_conv_filters,
                kernel_size=conv_filter_size,
                strides=1,
                name='convolution_' + str(i + 1)
            )(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(pool_size=2)(layer)
    
    layer = Dropout(0.5)(layer)
    #lstm blocks
    for j in range(n_lstm_blocks):
        layer = LSTM(n_lstm_units, 
                     return_sequences=True, 
                     name='lstm_' + str(j+1)
            )(layer)
        
    layer = Dropout(0.5)(layer)
    layer = TimeDistributed(Dense(10))(layer)
    layer = Activation('softmax', name='output_realtime')(layer)
    time_distributed_merge_layer = Lambda(
            function=lambda x: K.mean(x, axis=1), 
            output_shape=lambda shape: (shape[0],) + shape[2:],
            name='output_merged'
        )
    model_output = time_distributed_merge_layer(layer)
    model = Model(model_input, model_output)
    return model

In [ ]:
def get_crnn_model_koz4k_bn(input_shape, n_conv_blocks, n_conv_filters, conv_filter_size, n_lstm_blocks, n_lstm_units):
    model_input = Input(input_shape, name='input')
    layer = model_input
    layer = BatchNormalization(axis=-1)(layer)
    #convolution blocks
    for i in range(n_conv_blocks):
        layer = Convolution1D(
                filters=n_conv_filters,
                kernel_size=conv_filter_size,
                strides=1,
                name='convolution_' + str(i + 1)
            )(layer)
        layer = BatchNormalization(axis=-1)(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(pool_size=2)(layer)
        
    layer = BatchNormalization(axis=-1)(layer)
    layer = Dropout(0.5)(layer)
    #lstm blocks
    for j in range(n_lstm_blocks):
        layer = LSTM(n_lstm_units, 
                     return_sequences=True, 
                     name='lstm_' + str(j+1)
            )(layer)
        layer = BatchNormalization(axis=-1)(layer)
    
    layer = BatchNormalization(axis=-1)(layer)
    layer = Dropout(0.5)(layer)
    layer = TimeDistributed(Dense(10))(layer)
    layer = Activation('softmax', name='output_realtime')(layer)
    time_distributed_merge_layer = Lambda(
            function=lambda x: K.mean(x, axis=1), 
            output_shape=lambda shape: (shape[0],) + shape[2:],
            name='output_merged'
        )
    model_output = time_distributed_merge_layer(layer)
    model = Model(model_input, model_output)
    return model

In [ ]:
def get_cnn_model_koz4k(input_shape, n_conv_blocks, n_conv_filters, conv_filter_size):
    model_input = Input(input_shape, name='input')
    layer = model_input
    #convolution blocks
    for i in range(n_conv_blocks):
        layer = Convolution1D(
                filters=n_conv_filters,
                kernel_size=conv_filter_size,
                strides=2,
                name='convolution_' + str(i + 1)
            )(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(pool_size=2)(layer)
    #global pooling bloks   
    averagePool = GlobalAveragePooling1D()(layer)
    maxPool = GlobalMaxPooling1D()(layer)
    layer = concatenate([averagePool, maxPool])
    layer = Dropout(rate=0.5)(layer)
    #dense layers
    layer = Dense(10)(layer)
    model_output = Activation('softmax')(layer)
    model = Model(model_input, model_output)
    return model

In [ ]:
def get_cnn_model_koz4k_bn(input_shape, n_conv_blocks, n_conv_filters, conv_filter_size):
    model_input = Input(input_shape, name='input')
    layer = model_input
    layer = BatchNormalization(axis=-1)(layer)
    #convolution blocks
    for i in range(n_conv_blocks):
        layer = Convolution1D(
                filters=n_conv_filters,
                kernel_size=conv_filter_size,
                strides=1,
                name='convolution_' + str(i + 1)
            )(layer)
        layer = BatchNormalization(axis=-1)(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(pool_size=2)(layer)
    #global pooling bloks   
    averagePool = GlobalAveragePooling1D()(layer)
    maxPool = GlobalMaxPooling1D()(layer)
    layer = concatenate([averagePool, maxPool])
    layer = BatchNormalization(axis=-1)(layer)
    #dense layers
    layer = Dropout(rate=0.5)(layer)
    layer = Dense(10)(layer)
    model_output = Activation('softmax')(layer)
    model = Model(model_input, model_output)
    return model

In [ ]:
def get_crnn_model_keun(input_shape):
    #use tansorflow as backend
    freq_axis = 1
    time_axis = 2
    channel_axis = 3
    # Input block
    model_input = Input(shape=input_shape, name='input')
    x = ZeroPadding2D(padding=(0, 37))(model_input)
    x = BatchNormalization(axis=freq_axis, name='bn_0_freq')(x)
    # Conv block 1
    x = Convolution2D(64, 3, 3, border_mode='same', name='conv1')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn1')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1')(x)
    x = Dropout(0.1, name='dropout1')(x)
    # Conv block 2
    x = Convolution2D(128, 3, 3, border_mode='same', name='conv2')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn2')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(3, 3), name='pool2')(x)
    x = Dropout(0.1, name='dropout2')(x)
    # Conv block 3
    x = Convolution2D(128, 3, 3, border_mode='same', name='conv3')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn3')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='pool3')(x)
    x = Dropout(0.1, name='dropout3')(x)
    # Conv block 4
    x = Convolution2D(128, 3, 3, border_mode='same', name='conv4')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn4')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='pool4')(x)
    x = Dropout(0.1, name='dropout4')(x)
    
    x = Reshape((K.int_shape(x)[2], K.int_shape(x)[3]))(x)
    
    # GRU block 1, 2
    x = GRU(32, return_sequences=True, name='gru1')(x)
    x = GRU(32, return_sequences=False, name='gru2')(x)
    x = Dropout(0.3)(x)
    #ouput
    x = Dense(50, activation='sigmoid', name='output')(x)
    #Create model
    model = Model(model_input, x)
    return model

In [ ]:
def get_cnn_model_keun(input_shape):
    #use tansorflow as backend
    freq_axis = 1
    time_axis = 2
    channel_axis = 3
    # Input block
    model_input = Input(shape=input_shape, name='input')
    x = BatchNormalization(axis=freq_axis, name='bn_0_freq')(model_input)
    # Conv block 1
    x = Convolution2D(64, 3, 3, border_mode='same', name='conv1')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn1')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 4), name='pool1')(x)
    # Conv block 2
    x = Convolution2D(128, 3, 3, border_mode='same', name='conv2')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn2')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 4), name='pool2')(x)
    # Conv block 3
    x = Convolution2D(128, 3, 3, border_mode='same', name='conv3')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn3')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 4), name='pool3')(x)
    # Conv block 4
    x = Convolution2D(128, 3, 3, border_mode='same', name='conv4')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn4')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(3, 5), name='pool4')(x)
    # Conv block 5
    x = Convolution2D(64, 3, 3, border_mode='same', name='conv5')(x)
    x = BatchNormalization(axis=channel_axis, mode=0, name='bn5')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 4), name='pool5')(x)
    # Output
    x = Flatten()(x)
    x = Dense(10, activation='sigmoid', name='output')(x)
    # Create model
    model = Model(model_input, x)
    return model

In [ ]:
def wavenet_block(n_conv_filters, conv_filter_size, dilation_rate):
    def f(input_):
        residual = input_
        
        tanh_out = Convolution1D(n_conv_filters, conv_filter_size, padding='same', 
                                 dilation_rate=dilation_rate)(input_)
        tanh_out = BatchNormalization(axis=-1)(tanh_out)
        tanh_out = Activation('tanh')(tanh_out)
        
        sigmoid_out = Convolution1D(n_conv_filters, conv_filter_size, padding='same', 
                                    dilation_rate=dilation_rate)(input_)
        sigmoid_out = BatchNormalization(axis=-1)(sigmoid_out)
        sigmoid_out = Activation('sigmoid')(sigmoid_out)
        
        merged = merge([tanh_out, sigmoid_out], mode='mul')
        skip_out = Convolution1D(n_conv_filters, 1, activation='relu')(merged)
        out = merge([skip_out, residual], mode='sum')
        return out, skip_out
    return f

In [ ]:
def get_wavenet_model(input_shape, n_wavenet_block, n_conv_filter, conv_filter_size, max_dilation_rate):
    model_input = Input(shape=input_shape, name='input')
    x = Convolution1D(n_conv_filter, 1, padding='same')(model_input)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('tanh')(x)
    
    skip = []
    for i in range(n_wavenet_block):
        for r in range(int(math.log2(max_dilation_rate))):
            x, s = wavenet_block(n_conv_filter, conv_filter_size, 2**r)(x)
            skip.append(s)
    
    x = merge(skip, mode='sum')
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    x = Convolution1D(n_conv_filter, 1, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    x = Convolution1D(n_conv_filter, 1, padding='same')(x)
    
    x = Flatten()(x)
    model_output = Dense(10, activation='softmax')(x)
    model = Model(model_input, model_output)
    return model

In [ ]:
def get_vgg_model_1D(input_shape, n_conv_blocks, n_conv_filters, conv_filter_size):
    model_input = Input(input_shape, name='input')
    layer = model_input
    #convolution blocks
    for i in range(n_conv_blocks):
        layer = Convolution1D(
                filters=n_conv_filters,
                kernel_size=conv_filter_size,
                strides=1,
                padding='same',
            )(layer)
        layer = Activation('relu')(layer)
        layer = Convolution1D(
                filters=n_conv_filters,
                kernel_size=conv_filter_size,
                strides=1,
                padding='same'
            )(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(pool_size=2)(layer)
    #global pooling bloks   
    averagePool = GlobalAveragePooling1D()(layer)
    maxPool = GlobalMaxPooling1D()(layer)
    layer = concatenate([averagePool, maxPool])
    layer = Dropout(rate=0.5)(layer)
    #dense layers
    layer = Dense(10)(layer)
    model_output = Activation('softmax')(layer)
    model = Model(model_input, model_output)
    return model

In [2]:
def get_multi_level_cnn_model(input_shape):
    model_input = Input(shape=input_shape)
    
    x = Convolution2D(32, 3, padding='same')(model_input)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    x_avg1 = GlobalAveragePooling2D()(x)
    
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    x_avg2 = GlobalAveragePooling2D()(x)
    
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x_avg3 = GlobalAveragePooling2D()(x)
    
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x_avg4 = GlobalAveragePooling2D()(x)
    
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x_avg5 = GlobalAveragePooling2D()(x)
    
    x_merge = concatenate([x_avg1, x_avg2, x_avg3, x_avg4, x_avg5], axis=-1)
    
    x = Dense(32, activation='relu')(x_merge)
    x = Dropout(0.2)(x)
    model_output = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=model_input, outputs=model_output)
    
    return model

In [3]:
def get_multi_level_cnn_model_2(input_shape):
    model_input = Input(shape=input_shape)
    
    x = Convolution2D(64, 3, padding='same')(model_input)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = Convolution2D(64, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    x_avg1 = GlobalAveragePooling2D()(x)
    
    x = Convolution2D(128, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = Convolution2D(128, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    x_avg2 = GlobalAveragePooling2D()(x)
    
    x = Convolution2D(256, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = Convolution2D(256, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    x_avg3 = GlobalAveragePooling2D()(x)
    
    x = Convolution2D(512, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = Convolution2D(512, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    x_avg4 = GlobalAveragePooling2D()(x)
    
    x_merge = concatenate([x_avg1, x_avg2, x_avg3, x_avg4], axis=-1)
    
    x = Dense(64, activation='relu')(x_merge)
    x = Dropout(0.2)(x)
    model_output = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=model_input, outputs=model_output)
    
    return model

In [4]:
def get_multi_level_cnn_model_3(input_shape):
    model_input = Input(shape=input_shape)
    
    x = Convolution2D(32, 3, padding='same')(model_input)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = Convolution2D(32, 3, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    
    x = Flatten()(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    model_output = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=model_input, outputs=model_output)
    
    return model

In [2]:
def get_densenet_demo(input_shape):
    model_input = Input(shape=input_shape)
    
    x = Convolution2D(32, 3, padding='same')(model_input)
    x = BatchNormalization(axis=-1)(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(4, 2))(x)
    
    x1 = Convolution2D(32, 3, padding='same')(x)
    x1 = BatchNormalization(axis=-1)(x1)
    x1 = ELU()(x1)
    x1 = concatenate([x, x1], axis=-1)
    x1 = AveragePooling2D(pool_size=(4, 2))(x1)
    
    x2 = Convolution2D(32, 3, padding='same')(x1)
    x2 = BatchNormalization(axis=-1)(x2)
    x2 = ELU()(x2)
    x2 = concatenate([x1, x2], axis=-1)
    x2 = AveragePooling2D(pool_size=(4, 2))(x2)
    
    x3 = Convolution2D(32, 3, padding='same')(x2)
    x3 = BatchNormalization(axis=-1)(x3)
    x3 = ELU()(x3)
    x3 = concatenate([x2, x3], axis=-1)
    x3 = AveragePooling2D(pool_size=(4, 2))(x3)
    
    x4 = Flatten()(x3)
    x4 = Dense(32, activation='relu')(x4)
    x4 = Dropout(0.2)(x4)
    model_output = Dense(10, activation='softmax')(x4)
    
    model = Model(inputs=model_input, outputs=model_output)
    
    return model

In [3]:
def label_onehot_encode(y):
    y_onehot = []
    y_unique = sorted(set(y))
    num_classes = len(y_unique)
    for label in y:
        tmp = [0]*num_classes
        encode_index = y_unique.index(label)
        tmp[encode_index] = 1
        y_onehot.append(tmp)
    return np.array(y_onehot)

In [4]:
def process_data_for_conv2D(X, resize_shape=None):
    X_conv2D = []
    for sample in X:
        sample = np.reshape(sample, newshape=(sample.shape[0], sample.shape[1], 1))
        if resize_shape:
            sample = resize(sample, output_shape=resize_shape)
        X_conv2D.append(sample)
    return np.array(X_conv2D, dtype=np.float32)

In [5]:
def train_val_test_split(X, y, train_size, val_size, test_size):
    X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=train_size, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=test_size/(test_size + val_size), stratify=y_val_test)
    return X_train, y_train, X_val, y_val, X_test, y_test

In [6]:
X_melspec = np.load('GTZAN/without_split_features/melspec_feature_1024.npy')
y = np.load('GTZAN/onehot_labels.npy')
print(X_melspec.shape)
print(y.shape)

(1000, 1293, 128)
(1000, 10)


In [7]:
X_melspec = process_data_for_conv2D(X_melspec)
X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X_melspec, y, train_size=0.8, val_size=0.1, test_size=0.1)
print(X_train.shape, X_val.shape, X_test.shape)

((800, 1293, 128, 1), (100, 1293, 128, 1), (100, 1293, 128, 1))


In [ ]:
#without data argumatent
model = get_densenet_demo(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
model.summary()

opt = Adam(lr=0.001)
lr_change = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=2, min_lr=0.000)
model.compile(
            loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer=opt)
model.fit(X_train, y_train, batch_size=16, epochs=200, 
          validation_data=(X_val, y_val), verbose=1,
          callbacks=[lr_change])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1293, 128, 1)  0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 1293, 128, 32) 320         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 1293, 128, 32) 128         conv2d_1[0][0]                   
____________________________________________________________________________________________________
elu_1 (ELU)                      (None, 1293, 128, 32) 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [14]:
width_shift_range = 0.3
height_shift_range = 0.3

#with data argument
train_datagen = ImageDataGenerator(
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
)

model = get_multi_level_cnn_model_3(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
model.summary()

opt = Adam(lr=0.001)
lr_change = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, min_lr=0.000)
model.compile(
            loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer=opt)

model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=4), steps_per_epoch=len(X_train)//4, 
                    epochs=1000, validation_data=(X_val, y_val), verbose=1, 
                    callbacks=[lr_change])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 647, 128, 1)   0                                            
____________________________________________________________________________________________________
conv2d_21 (Conv2D)               (None, 647, 128, 32)  320         input_5[0][0]                    
____________________________________________________________________________________________________
batch_normalization_21 (BatchNor (None, 647, 128, 32)  128         conv2d_21[0][0]                  
____________________________________________________________________________________________________
elu_21 (ELU)                     (None, 647, 128, 32)  0           batch_normalization_21[0][0]     
___________________________________________________________________________________________

800/800 [==============================] - 25s - loss: 0.4971 - acc: 0.8219 - val_loss: 0.6673 - val_acc: 0.7800
Epoch 75/1000
800/800 [==============================] - 25s - loss: 0.4839 - acc: 0.8325 - val_loss: 0.4921 - val_acc: 0.8300
Epoch 76/1000
800/800 [==============================] - 25s - loss: 0.4794 - acc: 0.8341 - val_loss: 0.6262 - val_acc: 0.7800
Epoch 77/1000
800/800 [==============================] - 25s - loss: 0.4812 - acc: 0.8366 - val_loss: 0.4807 - val_acc: 0.8500
Epoch 78/1000
800/800 [==============================] - 25s - loss: 0.4580 - acc: 0.8381 - val_loss: 0.5554 - val_acc: 0.8200
Epoch 79/1000
800/800 [==============================] - 25s - loss: 0.4528 - acc: 0.8441 - val_loss: 0.6819 - val_acc: 0.8400
Epoch 80/1000
800/800 [==============================] - 25s - loss: 0.4610 - acc: 0.8375 - val_loss: 0.8453 - val_acc: 0.7500
Epoch 81/1000
800/800 [==============================] - 25s - loss: 0.4284 - acc: 0.8516 - val_loss: 0.7047 - val_acc: 0.840

800/800 [==============================] - 25s - loss: 0.2858 - acc: 0.9037 - val_loss: 0.5416 - val_acc: 0.8600
Epoch 203/1000
800/800 [==============================] - 25s - loss: 0.3007 - acc: 0.8931 - val_loss: 0.5413 - val_acc: 0.8600
Epoch 204/1000
800/800 [==============================] - 25s - loss: 0.3041 - acc: 0.8984 - val_loss: 0.5364 - val_acc: 0.8600
Epoch 205/1000
800/800 [==============================] - 25s - loss: 0.3262 - acc: 0.8837 - val_loss: 0.5371 - val_acc: 0.8600
Epoch 206/1000
800/800 [==============================] - 25s - loss: 0.3076 - acc: 0.8916 - val_loss: 0.5451 - val_acc: 0.8600
Epoch 207/1000
800/800 [==============================] - 24s - loss: 0.2876 - acc: 0.9000 - val_loss: 0.5489 - val_acc: 0.8600
Epoch 208/1000
800/800 [==============================] - 25s - loss: 0.3235 - acc: 0.8909 - val_loss: 0.5533 - val_acc: 0.8600
Epoch 209/1000
800/800 [==============================] - 25s - loss: 0.2938 - acc: 0.8991 - val_loss: 0.5405 - val_acc

800/800 [==============================] - 24s - loss: 0.2975 - acc: 0.8912 - val_loss: 0.5436 - val_acc: 0.8600
Epoch 331/1000
800/800 [==============================] - 25s - loss: 0.2943 - acc: 0.8984 - val_loss: 0.5458 - val_acc: 0.8600
Epoch 332/1000
800/800 [==============================] - 24s - loss: 0.3050 - acc: 0.8953 - val_loss: 0.5409 - val_acc: 0.8600
Epoch 333/1000
800/800 [==============================] - 25s - loss: 0.3210 - acc: 0.8881 - val_loss: 0.5413 - val_acc: 0.8600
Epoch 334/1000
800/800 [==============================] - 25s - loss: 0.3097 - acc: 0.8931 - val_loss: 0.5477 - val_acc: 0.8600
Epoch 335/1000
800/800 [==============================] - 25s - loss: 0.3114 - acc: 0.8875 - val_loss: 0.5270 - val_acc: 0.8600
Epoch 336/1000
800/800 [==============================] - 25s - loss: 0.3009 - acc: 0.8972 - val_loss: 0.5375 - val_acc: 0.8600
Epoch 337/1000
800/800 [==============================] - 24s - loss: 0.2962 - acc: 0.9006 - val_loss: 0.5431 - val_acc

800/800 [==============================] - 24s - loss: 0.3000 - acc: 0.8897 - val_loss: 0.5395 - val_acc: 0.8600
Epoch 459/1000
800/800 [==============================] - 24s - loss: 0.3132 - acc: 0.8937 - val_loss: 0.5493 - val_acc: 0.8600
Epoch 460/1000
800/800 [==============================] - 24s - loss: 0.3045 - acc: 0.8959 - val_loss: 0.5395 - val_acc: 0.8600
Epoch 461/1000
800/800 [==============================] - 24s - loss: 0.3093 - acc: 0.8881 - val_loss: 0.5437 - val_acc: 0.8600
Epoch 462/1000
800/800 [==============================] - 24s - loss: 0.3031 - acc: 0.8950 - val_loss: 0.5488 - val_acc: 0.8600
Epoch 463/1000
800/800 [==============================] - 24s - loss: 0.3041 - acc: 0.8891 - val_loss: 0.5378 - val_acc: 0.8600
Epoch 464/1000
800/800 [==============================] - 24s - loss: 0.2946 - acc: 0.9097 - val_loss: 0.5469 - val_acc: 0.8600
Epoch 465/1000
800/800 [==============================] - 24s - loss: 0.3084 - acc: 0.8978 - val_loss: 0.5410 - val_acc

800/800 [==============================] - 24s - loss: 0.3264 - acc: 0.8862 - val_loss: 0.5421 - val_acc: 0.8600
Epoch 587/1000
800/800 [==============================] - 24s - loss: 0.3101 - acc: 0.8906 - val_loss: 0.5438 - val_acc: 0.8600
Epoch 588/1000
800/800 [==============================] - 24s - loss: 0.3126 - acc: 0.8941 - val_loss: 0.5504 - val_acc: 0.8600
Epoch 589/1000
800/800 [==============================] - 24s - loss: 0.3112 - acc: 0.8944 - val_loss: 0.5534 - val_acc: 0.8600
Epoch 590/1000
800/800 [==============================] - 24s - loss: 0.3003 - acc: 0.8956 - val_loss: 0.5468 - val_acc: 0.8600
Epoch 591/1000
800/800 [==============================] - 24s - loss: 0.3010 - acc: 0.8959 - val_loss: 0.5395 - val_acc: 0.8600
Epoch 592/1000
800/800 [==============================] - 24s - loss: 0.3084 - acc: 0.8875 - val_loss: 0.5363 - val_acc: 0.8600
Epoch 593/1000
800/800 [==============================] - 24s - loss: 0.3213 - acc: 0.8856 - val_loss: 0.5486 - val_acc

800/800 [==============================] - 23s - loss: 0.3162 - acc: 0.8884 - val_loss: 0.5360 - val_acc: 0.8600
Epoch 715/1000
800/800 [==============================] - 23s - loss: 0.2815 - acc: 0.9056 - val_loss: 0.5417 - val_acc: 0.8600
Epoch 716/1000
800/800 [==============================] - 23s - loss: 0.3124 - acc: 0.8966 - val_loss: 0.5297 - val_acc: 0.8600
Epoch 717/1000
800/800 [==============================] - 23s - loss: 0.3129 - acc: 0.8903 - val_loss: 0.5358 - val_acc: 0.8600
Epoch 718/1000
800/800 [==============================] - 23s - loss: 0.3093 - acc: 0.8931 - val_loss: 0.5422 - val_acc: 0.8600
Epoch 719/1000
800/800 [==============================] - 23s - loss: 0.3127 - acc: 0.8912 - val_loss: 0.5384 - val_acc: 0.8600
Epoch 720/1000
800/800 [==============================] - 23s - loss: 0.3025 - acc: 0.8928 - val_loss: 0.5429 - val_acc: 0.8600
Epoch 721/1000
800/800 [==============================] - 23s - loss: 0.3076 - acc: 0.8950 - val_loss: 0.5418 - val_acc

800/800 [==============================] - 23s - loss: 0.2788 - acc: 0.9041 - val_loss: 0.5479 - val_acc: 0.8600
Epoch 843/1000
800/800 [==============================] - 23s - loss: 0.3079 - acc: 0.8934 - val_loss: 0.5412 - val_acc: 0.8600
Epoch 844/1000
800/800 [==============================] - 23s - loss: 0.3222 - acc: 0.8787 - val_loss: 0.5403 - val_acc: 0.8600
Epoch 845/1000
800/800 [==============================] - 23s - loss: 0.2930 - acc: 0.9000 - val_loss: 0.5378 - val_acc: 0.8600
Epoch 846/1000
800/800 [==============================] - 23s - loss: 0.3000 - acc: 0.8956 - val_loss: 0.5444 - val_acc: 0.8600
Epoch 847/1000
800/800 [==============================] - 23s - loss: 0.3165 - acc: 0.8866 - val_loss: 0.5397 - val_acc: 0.8600
Epoch 848/1000
800/800 [==============================] - 23s - loss: 0.3182 - acc: 0.8884 - val_loss: 0.5298 - val_acc: 0.8600
Epoch 849/1000
800/800 [==============================] - 23s - loss: 0.3011 - acc: 0.8959 - val_loss: 0.5369 - val_acc

800/800 [==============================] - 21s - loss: 0.3100 - acc: 0.8928 - val_loss: 0.5361 - val_acc: 0.8600
Epoch 971/1000
800/800 [==============================] - 21s - loss: 0.2869 - acc: 0.9022 - val_loss: 0.5436 - val_acc: 0.8600
Epoch 972/1000
800/800 [==============================] - 21s - loss: 0.3026 - acc: 0.8962 - val_loss: 0.5448 - val_acc: 0.8500
Epoch 973/1000
800/800 [==============================] - 21s - loss: 0.3054 - acc: 0.8944 - val_loss: 0.5480 - val_acc: 0.8600
Epoch 974/1000
800/800 [==============================] - 21s - loss: 0.2980 - acc: 0.8969 - val_loss: 0.5481 - val_acc: 0.8600
Epoch 975/1000
800/800 [==============================] - 21s - loss: 0.3035 - acc: 0.8997 - val_loss: 0.5481 - val_acc: 0.8600
Epoch 976/1000
800/800 [==============================] - 21s - loss: 0.3155 - acc: 0.8937 - val_loss: 0.5398 - val_acc: 0.8600
Epoch 977/1000
800/800 [==============================] - 21s - loss: 0.2867 - acc: 0.9000 - val_loss: 0.5358 - val_acc